## Spark DataFrame

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

In [2]:
from google.colab import files
files.upload()

Saving people.json to people.json


{'people.json': b'{"name":"Michael"}\n{"name":"Andy", "age":30}\n{"name":"Justin", "age":19}\n'}

In [9]:
# Read `people.json`
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadJson").getOrCreate()
json_file_path = "people.json"
df = spark.read.json(json_file_path)

df.show(truncate=False)

+----+-------+
|age |name   |
+----+-------+
|null|Michael|
|30  |Andy   |
|19  |Justin |
+----+-------+



In [12]:
# Show head
df.show(5)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [14]:
# Show schema
df.printSchema()


root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [15]:
# Show column
df.columns

['age', 'name']

In [16]:
# Describe
df.describe()

DataFrame[summary: string, age: string, name: string]

In [17]:
# Describe and show
df.describe().show()


+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [63]:
from pyspark.sql.types import (StructField, StringType,
                               IntegerType, StructType)

In [64]:
#age: name of variable, type of variable, True/False: if it can contain null
data_schema = [StructField('age', IntegerType(), True),
               StructField('name', StringType(), True)]

In [65]:
data_schema

[StructField(age,IntegerType,true), StructField(name,StringType,true)]

In [66]:
final_struc = StructType(fields=data_schema)

In [67]:
final_struc

StructType(List(StructField(age,IntegerType,true),StructField(name,StringType,true)))

In [68]:
df = spark.read.json('people.json', schema=final_struc)

In [69]:
df.printSchema()
#Now it has corrected the age as integer instead of long

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [70]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [71]:
df['age']

Column<'age'>

In [72]:
type(df['age'])

pyspark.sql.column.Column

In [73]:
#If you want to select a dataframe column
df.select('age')

DataFrame[age: int]

In [74]:
#If you want to view the results of dataframe column
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [75]:
#If you want to view selected rows of dataframe
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [76]:
df.head(2)[0]

Row(age=None, name='Michael')

In [77]:
df.head(2)[0]['name']

'Michael'

In [78]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [79]:
type(df.head(2)[0]['name'])

str

In [ ]:
##The reason there are so many datatypes and objects in Spark because of its ability
#to read from a distributed source and map that out to compute

In [80]:
#selecting multiple columns
df.select('age')

DataFrame[age: int]

In [81]:
# Show the dataframe
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [82]:
#Creating new columns `newAge`
from pyspark.sql.functions import col
df1 = df.withColumn("newAge", col('age'))
df1.show()

+----+-------+------+
| age|   name|newAge|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [84]:
#Creating new columns `doubleAge`
df2 = df.withColumn("doubleAge", col('age') *2)
df2.show()


+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [ ]:
#The above operations are not inplace so everytime you make
#any changes you have to save them to a new variable

In [85]:
#Renaming a column 'age' by 'my_new_age'
df3 =df.withColumnRenamed("age", "my_new_age")
df3.show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



## Working with Spark SQL queries

In [86]:
#In order to work with SQL queries we need to register spark
#As a temporary SQL view
#We do CreateOrReplace incase we have already existing view it replaces

df.createOrReplaceTempView('people')

In [87]:
#Now you can pass direct SQL queries
results = spark.sql("SELECT * FROM people")

In [88]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [90]:
# Select people with age = 30
df4 = df.filter(df["age"] == 30)

In [91]:
# Show
df4.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## Working with JSON, CSV and other formats

In [92]:
from pyspark.sql import SparkSession

In [93]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [94]:
from google.colab import files
files.upload()

Saving appl_stock.csv to appl_stock.csv


{'appl_stock.csv': b'Date,Open,High,Low,Close,Volume,Adj Close\n2010-01-04,213.429998,214.499996,212.38000099999996,214.009998,123432400,27.727039\n2010-01-05,214.599998,215.589994,213.249994,214.379993,150476200,27.774976000000002\n2010-01-06,214.379993,215.23,210.750004,210.969995,138040000,27.333178000000004\n2010-01-07,211.75,212.000006,209.050005,210.58,119282800,27.28265\n2010-01-08,210.299994,212.000006,209.06000500000002,211.98000499999998,111902700,27.464034\n2010-01-11,212.79999700000002,213.000002,208.450005,210.11000299999998,115557400,27.221758\n2010-01-12,209.18999499999998,209.76999500000002,206.419998,207.720001,148614900,26.91211\n2010-01-13,207.870005,210.92999500000002,204.099998,210.650002,151473000,27.29172\n2010-01-14,210.11000299999998,210.45999700000002,209.020004,209.43,108223500,27.133657\n2010-01-15,210.92999500000002,211.59999700000003,205.869999,205.93,148516900,26.680197999999997\n2010-01-19,208.330002,215.18999900000003,207.240004,215.039995,182501900,27.

In [95]:
#infer schema option is available with CSV
df = spark.read.csv('/content/appl_stock.csv', inferSchema=True, header=True)
df.show(truncate=False)


+----------+------------------+------------------+------------------+------------------+---------+------------------+
|Date      |Open              |High              |Low               |Close             |Volume   |Adj Close         |
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998        |214.499996        |212.38000099999996|214.009998        |123432400|27.727039         |
|2010-01-05|214.599998        |215.589994        |213.249994        |214.379993        |150476200|27.774976000000002|
|2010-01-06|214.379993        |215.23            |210.750004        |210.969995        |138040000|27.333178000000004|
|2010-01-07|211.75            |212.000006        |209.050005        |210.58            |119282800|27.28265          |
|2010-01-08|210.299994        |212.000006        |209.06000500000002|211.98000499999998|111902700|27.464034         |
|2010-01-11|212.79999700000002|213.000002        |208.45

In [97]:
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType, IntegerType


In [101]:
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType, IntegerType
schema = StructType([
    StructField("Date", TimestampType(), True),
    StructField("Open", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Low", DoubleType(), True),
    StructField("Close", DoubleType(), True),
    StructField("Volume", IntegerType(), True),
    StructField("Adj Close", DoubleType(), True)
])

df.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [102]:
# Head 3
df.show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [103]:
%%time
df.head(1)[0]

CPU times: user 4 ms, sys: 2.09 ms, total: 6.09 ms
Wall time: 164 ms


Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [104]:
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [116]:
#You can pass in conditions 'Close < 500'
x = df.filter(df["Close"] < 500)
close_price = (x.select("Close"))
close_price.show()

+------------------+
|             Close|
+------------------+
|        214.009998|
|        214.379993|
|        210.969995|
|            210.58|
|211.98000499999998|
|210.11000299999998|
|        207.720001|
|        210.650002|
|            209.43|
|            205.93|
|        215.039995|
|            211.73|
|        208.069996|
|            197.75|
|        203.070002|
|        205.940001|
|        207.880005|
|        199.289995|
|        192.060003|
|        194.729998|
+------------------+
only showing top 20 rows



In [117]:
#Opening price of every stock where Close price is less than 500
y = df.filter(df["Close"] < 500)
open_prices = y.select("Open")
open_prices.show()

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



In [119]:
# Select and show 'Open' and 'Close' with 'Close < 500'
selected_prices = df.filter(df["Close"] < 500).select("Open", "Close")
selected_prices.show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [120]:
# Select and show 'Open' and 'Close' with 'Open < 500'
selected_prices = df.filter(df["Open"] < 500).select("Open", "Close")
selected_prices.show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [121]:
# Select and show 'Volume' with 'Close < 500'
open_prices = y.select("Volume")
open_prices.show()

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
|115557400|
|148614900|
|151473000|
|108223500|
|148516900|
|182501900|
|153038200|
|152038600|
|220441900|
|266424900|
|466777500|
|430642100|
|293375600|
|311488100|
|187469100|
+---------+
only showing top 20 rows



In [124]:
# Multiple conditions 'Close' < 200  and 'Open' > 200
filtered_df = df.filter((df["Close"] < 200) & (df["Open"] > 200))
filtered_df.show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [125]:
# Multiple conditions 'Close' < 200  and 'Open' <= 200
z= df.filter((df["Close"] < 200) & (df["Open"] <= 200))

# Show the DataFrame with filtered rows
z.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|        194.420004|        199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|        198.370001|        191.570005|        192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|             196.0|        190.850002|        195.460001|212576700|25.323710000000002|
|2010-02-08|        195.690006|197.88000300000002|      

In [126]:
df.filter(df['Low'] == 197.16).show()

+----------+------------------+----------+------+------+---------+---------+
|      Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+----------+------------------+----------+------+------+---------+---------+
|2010-01-22|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+----------+------------------+----------+------+------+---------+---------+



## GroupBy, Aggregate and Sorting Functions

In [127]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [128]:
from google.colab import files
files.upload()

Saving sales_info.csv to sales_info.csv


{'sales_info.csv': b'Company,Person,Sales\nGOOG,Sam,200\nGOOG,Charlie,120\nGOOG,Frank,340\nMSFT,Tina,600\nMSFT,Amy,124\nMSFT,Vanessa,243\nFB,Carl,870\nFB,Sarah,350\nAPPL,John,250\nAPPL,Linda, 130\nAPPL,Mike, 750\nAPPL, Chris, 350'}

In [129]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [130]:
df.count()

12

In [131]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [132]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [139]:
# groupBy 'Company'and show the counting
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
df1 = df.groupBy("Company").agg(count("*").alias("Count"))
df1.show()

+-------+-----+
|Company|Count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [151]:
# groupBy 'Company'and show the mean
from pyspark.sql.functions import mean, max, min,sum
from pyspark.sql.types import DoubleType
df2 = df.groupBy("Company").agg(mean("Sales").alias("avg(Sales)"))
df2.show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [141]:
# groupBy 'Company'and show the max
df3 = df.groupBy("Company").agg(max("Sales").alias("max(Sales)"))
df3.show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [142]:
# groupBy 'Company'and show the min
df4 = df.groupBy("Company").agg(min("Sales").alias("min(Sales)"))
df4.show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [154]:
average_sales_per_company = (
    df.groupBy("Company")
    .agg(mean("Sales").alias("Average_Sales"))
    .orderBy("Company")
)
average_sales_per_company.show()

+-------+-----------------+
|Company|    Average_Sales|
+-------+-----------------+
|   APPL|            370.0|
|     FB|            610.0|
|   GOOG|            220.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [156]:
## May be you dont need group by, you need something
#like average sales per company
sum_of_average_sales = average_sales_per_company.agg(sum("Average_Sales").alias("Sum_Of_Average_Sales"))

# Show the resulting DataFrame
print("Sum of Average Sales per Company:")
sum_of_average_sales.show()


Sum of Average Sales per Company:
+--------------------+
|Sum_Of_Average_Sales|
+--------------------+
|  1522.3333333333333|
+--------------------+



In [157]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [159]:
#Counts distinct sales value
distinct_sales_count = df.agg(countDistinct("Sales").alias("count(DISTINCT Sales)"))
distinct_sales_count.show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [160]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [163]:
#Alias - Give the column name appropriate name
average_sales = df.select(avg('Sales').alias("Average Sales"))
average_sales.show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [164]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [165]:
#Formatting the number
from pyspark.sql.functions import format_number

In [166]:
sales_std = df.select(stddev('Sales').alias('std'))

In [167]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [173]:
#number of decimal places I want to show
formatted_sales_std = sales_std.select(format_number("std", 2).alias("format_number(std, 2)"))
formatted_sales_std.show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [ ]:
#Ordering the columns, default is ascending order


+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [ ]:
#Ordering in descending


+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Dealing with Missing Data in Spark

In [179]:
spark = SparkSession.builder.appName('miss').getOrCreate()

In [180]:
from google.colab import files
files.upload()

Saving ContainsNull.csv to ContainsNull.csv


{'ContainsNull.csv': b'Id,Name,Sales\nemp1,John,\nemp2,,\nemp3,,345.0\nemp4,Cindy,456.0\n'}

In [181]:
df = spark.read.csv('ContainsNull.csv', header=True, inferSchema=True)

In [182]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [184]:
#You can drop all the missing data
df_no_missing_data = df.dropna()
df_no_missing_data.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [185]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [186]:
#When we specify thresh=2, it looks at the row if it contains more than 2 non-null values, it drops
df_thresh_2 = df.dropna(thresh=2)
df_thresh_2.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [188]:
#By default how is set to 'any', meaning drop if there are any null values
df_no_missing_data.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [189]:
#We can change the how to 'all', if row has all null values only then drop

df_drop_all_null = df.dropna(how='all')
df_drop_all_null.show()


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [190]:
#The subset method will only drop if Sales colum has null values
df_drop_sales_null = df.dropna(subset=["Sales"])
df_drop_sales_null.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [191]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [194]:
df_filled = df.fillna(value='FILL VALUE', subset=["Sales"])

# Show the resulting DataFrame
df_filled.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [195]:
#We can also fill NULL values
df_filled = df.fillna(value="FILL VALUE", subset=["Sales"])

# Show the resulting DataFrame
df_filled.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [196]:
#Spark is smart enough to recognize string datatype and only fill
#in string datatype column, similar thing it does for numeric values
df_filled.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [197]:
from pyspark.sql.functions import mean

In [198]:
mean_val = df.select(mean(df['Sales'])).collect()

In [199]:
mean_val[0]

Row(avg(Sales)=400.5)

In [200]:
mean_val[0][0]

400.5

In [201]:
mean_sales = mean_val[0][0]

In [202]:
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [203]:
#Doing all this in one line
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Dates and Timestamps

In [204]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [205]:
df = spark.read.csv('appl_stock.csv', header=True, inferSchema=True)

In [206]:
df.count()

1762

In [207]:
df.head(1)
#datetime object in Date column, this looks messy here

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [208]:
df.select(['Date', 'Open']).show()

+----------+------------------+
|      Date|              Open|
+----------+------------------+
|2010-01-04|        213.429998|
|2010-01-05|        214.599998|
|2010-01-06|        214.379993|
|2010-01-07|            211.75|
|2010-01-08|        210.299994|
|2010-01-11|212.79999700000002|
|2010-01-12|209.18999499999998|
|2010-01-13|        207.870005|
|2010-01-14|210.11000299999998|
|2010-01-15|210.92999500000002|
|2010-01-19|        208.330002|
|2010-01-20|        214.910006|
|2010-01-21|        212.079994|
|2010-01-22|206.78000600000001|
|2010-01-25|202.51000200000001|
|2010-01-26|205.95000100000001|
|2010-01-27|        206.849995|
|2010-01-28|        204.930004|
|2010-01-29|        201.079996|
|2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows



In [209]:
from pyspark.sql.functions import (dayofmonth, hour,
                                   dayofyear, month,
                                   year, weekofyear,
                                  format_number, date_format)

In [210]:
#Whenever you are using any pyspark imported function,
#you can directly use them in select on your column
df.select(dayofmonth(df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [211]:
df.select(hour(df['Date'])).show()

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [212]:
df.select(month(df['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [217]:
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [222]:
from pyspark.sql.functions import col, year, avg

columns = ["Date", "Close"]

# Convert the "Date" column to a date type
df = df.withColumn("Date", col("Date").cast("date"))

# Extract the year from the "Date" column using the year function
df = df.withColumn("Year", year("Date"))

# Group by year and calculate the average closing price per year
average_closing_per_year = (
    df.groupBy("Year")
    .agg(avg("Close").alias("AverageClosingPrice"))
    .orderBy("Year")
)

# Show the resulting DataFrame
average_closing_per_year.show()

+----+-------------------+
|Year|AverageClosingPrice|
+----+-------------------+
|2010|  259.8424600000002|
|2011| 364.00432532142867|
|2012|  576.0497195640002|
|2013|  472.6348802857143|
|2014|  295.4023416507935|
|2015| 120.03999980555547|
|2016| 104.60400786904763|
+----+-------------------+



In [224]:
## May be you want to know average closing price per year
from pyspark.sql.functions import col, year, avg
columns = ["Date", "Close"]
df = df.withColumn("Date", col("Date").cast("date"))
df = df.withColumn("Year", year("Date"))
average_closing_per_year = (
    df.groupBy("Year")
    .agg(avg("Close").alias("year(Date)"))
    .orderBy("Year")
)
average_closing_per_year.show()


+----+------------------+
|Year|        year(Date)|
+----+------------------+
|2010| 259.8424600000002|
|2011|364.00432532142867|
|2012| 576.0497195640002|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2015|120.03999980555547|
|2016|104.60400786904763|
+----+------------------+



In [226]:
# Take the year
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
|2010-01-11|212.

In [230]:
df = df.withColumn("Year", year("Date"))

# Group by year and calculate the average values
average_per_year = (
    df.groupBy("Year")
    .agg(
        avg("Open").alias("avg(Open)"),
        avg("High").alias("avg(High)"),
        avg("Low").alias("avg(Low)"),
        avg("Close").alias("avg(Close)"),
        avg("Volume").alias("avg(Volume)"),
        avg("Adj Close").alias("avg(Adj Close)"),
        avg("Year").alias("avg(Year)")
    )
    .orderBy("Year")
)

# Show the resulting DataFrame
average_per_year.show(truncate=False)

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|avg(Open)         |avg(High)         |avg(Low)          |avg(Close)        |avg(Volume)         |avg(Adj Close)    |avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2010|259.9576190992064 |262.36880881349214|256.84761791269847|259.8424600000002 |1.4982631666666666E8|33.665072424603196|2010.0   |
|2011|364.06142773412705|367.4235704880951 |360.29769878174613|364.00432532142867|1.2307474166666667E8|47.16023692063492 |2011.0   |
|2012|576.652720788     |581.8254008040001 |569.9211606079999 |576.0497195640002 |1.319642044E8       |74.81383696800002 |2012.0   |
|2013|473.1281355634922 |477.6389272301587 |468.24710264682557|472.6348802857143 |1.016087E8          |62.61798788492063 |2013.0   |
|2014|295.1426195357143 |297.56103184523823|292.9949599801587 |295.40

In [231]:
# Create new column `Year`
df = df.withColumn("Year", year("Date"))
average_per_year = (
    df.groupBy("Year")
    .agg(
        avg("Open").alias("avg(Open)"),
        avg("High").alias("avg(High)"),
        avg("Low").alias("avg(Low)"),
        avg("Close").alias("avg(Close)"),
        avg("Volume").alias("avg(Volume)"),
        avg("Adj Close").alias("avg(Adj Close)"),
        avg("Year").alias("avg(Year)")
    )
    .orderBy("Year")
)

# Show the resulting DataFrame
average_per_year.show(truncate=False)

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|avg(Open)         |avg(High)         |avg(Low)          |avg(Close)        |avg(Volume)         |avg(Adj Close)    |avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2010|259.9576190992064 |262.36880881349214|256.84761791269847|259.8424600000002 |1.4982631666666666E8|33.665072424603196|2010.0   |
|2011|364.06142773412705|367.4235704880951 |360.29769878174613|364.00432532142867|1.2307474166666667E8|47.16023692063492 |2011.0   |
|2012|576.652720788     |581.8254008040001 |569.9211606079999 |576.0497195640002 |1.319642044E8       |74.81383696800002 |2012.0   |
|2013|473.1281355634922 |477.6389272301587 |468.24710264682557|472.6348802857143 |1.016087E8          |62.61798788492063 |2013.0   |
|2014|295.1426195357143 |297.56103184523823|292.9949599801587 |295.40

In [229]:
# Groupby `Year` and average the Close price
average_closing_per_year.show()


+----+-------------------+
|Year|AverageClosingPrice|
+----+-------------------+
|2010|  259.8424600000002|
|2011| 364.00432532142867|
|2012|  576.0497195640002|
|2013|  472.6348802857143|
|2014|  295.4023416507935|
|2015| 120.03999980555547|
|2016| 104.60400786904763|
+----+-------------------+



## Thank you!